# Random Forest Bellwether Classifier

## Import Libraries

In [292]:
import numpy as np
import pandas as pd
import math
import matplotlib.pyplot as plt
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, BaggingClassifier
from sklearn import metrics
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report
from sklearn import svm
from sklearn.svm import SVC
from sklearn import model_selection
from numpy import mean
from numpy import std
from sklearn.datasets import make_regression
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import RepeatedKFold
from sklearn.ensemble import BaggingRegressor
from collections import Counter
from sklearn.datasets import make_classification
from imblearn.over_sampling import RandomOverSampler
import statistics

# check version number
import imblearn
print(imblearn.__version__)

0.8.0


## Load Dataset

In [293]:
data = pd.read_csv('data/data_cnd.csv')
data = data.drop(['Unnamed: 0'], axis=1)

In [294]:
data.head()

fips    state   county  wins  poverty_rate     population  \
0  1001  Alabama  Autauga     3     21.933333   34419.166667   
1  1003  Alabama  Baldwin     3     19.716667  100428.166667   
2  1005  Alabama  Barbour     3     36.383333   24581.000000   
3  1007  Alabama     Bibb     3     29.083333   17001.000000   
4  1009  Alabama   Blount     3     21.583333   39136.666667   

   persons_in_poverty  rural_urban_code  urban_influence_code  metro  ...  \
0         6205.166667                 4                     4      0  ...   
1        16443.333333                 4                     4      0  ...   
2         8902.166667                 4                     4      0  ...   
3         4679.500000                 4                     4      0  ...   
4         7474.666667                 4                     4      0  ...   

   percent_black_male_2019  percent_black_female_2019  \
0                 9.373714                  10.739408   
1                 4.249353                   4.527536   
2                25.565098                  22.664668   
3                13.132982                   8.136108   
4                 0.892332                   0.798949   

   percent_native_american_male_2019  percent_native_american_female_2019  \
0                           0.216578                             0.259536   
1                           0.404508                             0.375839   
2                           0.417241                             0.271409   
3                           0.236671                             0.223274   
4                           0.332031                             0.307820   

   percent_asian_male_2019  percent_asian_female_2019  \
0                 0.511912                   0.662264   
1                 0.417499                   0.648647   
2                 0.222798                   0.247104   
3                 0.102706                   0.111637   
4                 0.146993                   0.172933   

   percent_hispanic_male_2019  percent_hispanic_female_2019  \
0                    1.582273                      1.408652   
1                    2.483941                      2.234875   
2                    2.548003                      1.976829   
3                    1.531660                      1.250335   
4                    5.101511                      4.551586   

   cumulative_confirmed_cases  cumulative_deaths  
0                      733294              10242  
1                     2279213              26929  
2                      298307               4787  
3                      295192               6094  
4                      715644              10437  

[5 rows x 44 columns]

## Transform Data

In [295]:
# Add 2016 bellwether code

l = []
for i, v in enumerate(data['wins']):
    if data.iloc[i, 3] >= 5:
        l.append(1)
    elif data.iloc[i, 3] < 5:
        l.append(0)
data['bellwether_2016'] = np.array(l, dtype=int)

In [296]:
# Add 2020 bellwether code

l = []
for i, v in enumerate(data['wins']):
    if data.iloc[i, 3] == 6:
        l.append(1)
    elif data.iloc[i, 3] < 6:
        l.append(0)
data['bellwether_2020'] = np.array(l, dtype=int)

In [297]:
# Add >4 wins code

l = []
for i, v in enumerate(data['wins']):
    if data.iloc[i, 3] >= 4:
        l.append(1)
    elif data.iloc[i, 3] < 4:
        l.append(0)
data['bellwether'] = np.array(l, dtype=int)

# General Information on Dataset

In [298]:
data.shape

(3108, 47)

In [299]:
data.columns

Index(['fips', 'state', 'county', 'wins', 'poverty_rate', 'population',
       'persons_in_poverty', 'rural_urban_code', 'urban_influence_code',
       'metro', 'civilian_labor_force', 'employed', 'unemployed',
       'unemployment_rate', 'no_diploma', 'diploma_only',
       'some_college/associate_degree', 'completed_college/bachelor_degree',
       'percent_no_diploma', 'percent_diploma_only',
       'percent_some_college/associate_degree',
       'percent_completed_college/bachelor_degree', 'percent_white_male_2010',
       'percent_white_female_2010', 'percent_black_male_2010',
       'percent_black_female_2010', 'percent_native_american_male_2010',
       'percent_native_american_female_2010', 'percent_asian_male_2010',
       'percent_asian_female_2010', 'percent_hispanic_male_2010',
       'percent_hispanic_female_2010', 'percent_white_male_2019',
       'percent_white_female_2019', 'percent_black_male_2019',
       'percent_black_female_2019', 'percent_native_american_male_2019

In [300]:
X = data.drop(['fips', 'state', 'county', 'wins', 'bellwether_2016', 'bellwether_2020', 'bellwether'], axis=1)
X.columns

Index(['poverty_rate', 'population', 'persons_in_poverty', 'rural_urban_code',
       'urban_influence_code', 'metro', 'civilian_labor_force', 'employed',
       'unemployed', 'unemployment_rate', 'no_diploma', 'diploma_only',
       'some_college/associate_degree', 'completed_college/bachelor_degree',
       'percent_no_diploma', 'percent_diploma_only',
       'percent_some_college/associate_degree',
       'percent_completed_college/bachelor_degree', 'percent_white_male_2010',
       'percent_white_female_2010', 'percent_black_male_2010',
       'percent_black_female_2010', 'percent_native_american_male_2010',
       'percent_native_american_female_2010', 'percent_asian_male_2010',
       'percent_asian_female_2010', 'percent_hispanic_male_2010',
       'percent_hispanic_female_2010', 'percent_white_male_2019',
       'percent_white_female_2019', 'percent_black_male_2019',
       'percent_black_female_2019', 'percent_native_american_male_2019',
       'percent_native_american_female_

In [301]:
X2=X[["poverty_rate", "population", "unemployment_rate", "percent_diploma_only", "percent_some_college/associate_degree", "percent_white_female_2010", "percent_white_male_2010", "percent_black_male_2010","percent_black_female_2010", "cumulative_confirmed_cases"]]
#X2=X[["poverty_rate"]]

In [302]:
X2.shape
X=X2

In [303]:
y = data.bellwether

In [304]:
X.shape

(3108, 10)

# Splitting into Train-Test Data

In [305]:
# split data into training and testing

from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = \
    train_test_split(X, y, test_size=0.25, random_state=0)

In [306]:
# general info on data
y_train.shape

(2331,)

In [307]:
# general info on data
X_train.shape

(2331, 10)

In [308]:
# general info on data
X_test.shape

(777, 10)

In [309]:
# printing dictionary of how many bellwethers and nonbellwethers there are in the train set -- needs to be fixed with oversampling
print(Counter(y_train))

Counter({0: 2094, 1: 237})


In [310]:
# printing dictionary of how many bellwethers and nonbellwethers there are in the test set
print(Counter(y_test))

Counter({0: 700, 1: 77})


# Balancing the Test Set

In [311]:
# define oversampling strategy - 2:1 ratio of nonbellwethers to bellwethers
oversample = RandomOverSampler(sampling_strategy=0.5)
# fit and apply the transform
X_over_test, y_over_test = oversample.fit_resample(X_test, y_test)
# summarize class distribution
print(Counter(y_over_test))

Counter({0: 700, 1: 350})


# Balacing the Train Set

In [312]:
# define oversampling strategy - 2:1 ratio of nonbellwethers to bellwethers
oversample = RandomOverSampler(sampling_strategy=0.5)
# fit and apply the transform
X_over, y_over = oversample.fit_resample(X_train, y_train)
# summarize class distribution
print(Counter(y_over))

Counter({0: 2094, 1: 1047})


In [283]:
model = RandomForestClassifier()

model.fit(X_over, y_over)

predicted_labels = model.predict(X_over_test)

#report=classification_report(y_over_test, predicted_labels)
#print(report)

metrics.confusion_matrix(y_over_test, predicted_labels)

#print (accuracy_score(y_over_test, predicted_labels))

array([[577, 123],
       [276,  74]])

In [313]:
#plt.figure(figsize=(25,30))
#plt.barh(X.columns, model.feature_importances_)
#print(model.feature_importances_)
X.columns

Index(['poverty_rate', 'population', 'unemployment_rate',
       'percent_diploma_only', 'percent_some_college/associate_degree',
       'percent_white_female_2010', 'percent_white_male_2010',
       'percent_black_male_2010', 'percent_black_female_2010',
       'cumulative_confirmed_cases'],
      dtype='object')

In [314]:
X_over.columns

Index(['poverty_rate', 'population', 'unemployment_rate',
       'percent_diploma_only', 'percent_some_college/associate_degree',
       'percent_white_female_2010', 'percent_white_male_2010',
       'percent_black_male_2010', 'percent_black_female_2010',
       'cumulative_confirmed_cases'],
      dtype='object')

In [315]:
X_over.shape

(3141, 10)

# Tuning the Hyperparameters

In [324]:
d_range = np.arange(1, 20, 2)
n_range = np.arange(50, 500, 50)

In [325]:
training_error = []
testing_error = []

for d in d_range:
    for n in n_range:
        rf_model = RandomForestClassifier(n_estimators=n,max_depth=d, 
                                          random_state=0)

        rf_model.fit(X_over, y_over)

        y_pred_class = rf_model.predict(X_over)
        training_accuracy = metrics.accuracy_score(y_over, y_pred_class)
        training_error.append(1 - training_accuracy)

        y_pred_class = rf_model.predict(X_over_test)
        testing_accuracy = metrics.accuracy_score(y_over_test, y_pred_class)
        testing_error.append(1 - testing_accuracy)

In [326]:
parameters_d = []
parameters_n = []

for d in d_range:
    for n in n_range:
        parameters_d.append(d)
        parameters_n.append(n)

In [327]:
# Create a DataFrame of the search with training error, and testing error.

column_dict = {'d': parameters_d, 'n': parameters_n, 'training error': training_error, 'testing error': testing_error}
df = pd.DataFrame(column_dict)
df.head()

d    n  training error  testing error
0  1   50        0.333333       0.333333
1  1  100        0.333333       0.333333
2  1  150        0.333333       0.333333
3  1  200        0.333333       0.333333
4  1  250        0.333333       0.333333

In [328]:
df = df.sort_values(by='testing error', ascending=True)
df.head()

d    n  training error  testing error
44   9  450        0.029608       0.315238
57  13  200        0.000637       0.317143
58  13  250        0.000637       0.321905
56  13  150        0.000637       0.323810
10   3  100        0.310729       0.323810

# Machine Learning Model Based on Adjusted Train-Test

In [316]:
attributes_importance={}
for i in X_over.columns:
    attributes_importance[i]=0

In [317]:
def remove_lowest_key():
    key=min(attributes_importance,key=attributes_importance.get)
    del attributes_importance[key]
    X=X.drop([key],axis=1)

In [329]:
#200 7
model=RandomForestClassifier(n_estimators=450,max_depth=9)
accuracy_list=[]

for i in range(30):
    model.fit(X_over,y_over)
    count=0
    y_pred_class = model.predict(X_over_test)
    report=classification_report(y_over_test, y_pred_class,output_dict=True)
    precision=report["1"]["precision"]
    accuracy_list.append(precision)
    for i in X_over.columns:
        attributes_importance[i]+=model.feature_importances_[count]
        count+=1
for i in X_over.columns:
    attributes_importance[i]=attributes_importance[i]/30

# for i in range(15):
#     attributes_importance=dict.fromkeys(attributes_importance,0)
#     for i in range(30):
#         model.fit(X_over,y_over)
#         count=0
#         for i in X_over.columns:
#             attributes_importance[i]+=model.feature_importances_[count]
#             count+=1
#     for i in X_over.columns:
#         attributes_importance[i]=attributes_importance[i]/30
#     key=min(attributes_importance,key=attributes_importance.get)
#     del attributes_importance[key]
#     X_over=X_over.drop([key],axis=1)
#     key=min(attributes_importance,key=attributes_importance.get)
#     del attributes_importance[key]
#     X_over=X_over.drop([key],axis=1)

   
    
    

In [333]:
y_over_test.shape

(1050,)

In [330]:
print(attributes_importance)

{'poverty_rate': 0.15226069001815687, 'population': 0.12141818863551486, 'unemployment_rate': 0.08226991827721043, 'percent_diploma_only': 0.11062785225034902, 'percent_some_college/associate_degree': 0.13762102439387044, 'percent_white_female_2010': 0.08873133594668156, 'percent_white_male_2010': 0.08172690496677151, 'percent_black_male_2010': 0.08810166330787286, 'percent_black_female_2010': 0.08596702114270793, 'cumulative_confirmed_cases': 0.08460873439419771}


In [249]:
sum=0
for i in X_over.columns:
    sum+=attributes_importance[i]
print(sum)

1.0


In [331]:
#average of accuracies from 30 runs of the model
statistics.mean(accuracy_list)

0.531994475368751

In [332]:
#standard deviation of 30 accuracies
statistics.stdev(accuracy_list)

0.021054450407158077

# Other Bagging Attempt (for testing purposes)

In [88]:
#bagged decision trees from https://machinelearningmastery.com/ensemble-machine-learning-algorithms-python-scikit-learn/
seed = 7
kfold = model_selection.KFold(n_splits=10, random_state=seed)
cart = DecisionTreeClassifier()
num_trees = 50
model = BaggingClassifier(base_estimator=cart, max_samples=200, max_features = 100, n_estimators=num_trees, random_state=seed)
results = model_selection.cross_val_score(model, X_train, y_train, cv=kfold)
print(results.mean())

0.9004484304932736


In [89]:
model.fit(X_train, y_train)

BaggingClassifier(base_estimator=DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=None,
            max_features=None, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, presort=False, random_state=None,
            splitter='best'),
         bootstrap=True, bootstrap_features=False, max_features=100,
         max_samples=200, n_estimators=50, n_jobs=1, oob_score=False,
         random_state=7, verbose=0, warm_start=False)

In [92]:
model.score(X_train,y_train)

0.9022421524663677

In [93]:
y_pred_class = model.predict(X_test)
testing_accuracy = metrics.accuracy_score(y_test, y_pred_class)
testing_error = 1 - testing_accuracy

In [94]:
metrics.confusion_matrix(y_test, y_pred_class)

array([[660,   2],
       [ 82,   0]])

In [46]:
#bg = BaggingClassifier(DecisionTreeClassifier(), max_samples=2230, max_features = 165, n_estimators=5)
bg = BaggingClassifier()

In [47]:
bg.fit(X_train,y_train)

BaggingClassifier(base_estimator=None, bootstrap=True,
         bootstrap_features=False, max_features=1.0, max_samples=1.0,
         n_estimators=10, n_jobs=1, oob_score=False, random_state=None,
         verbose=0, warm_start=False)

In [48]:
bg.score(X_test,y_test)

0.8817204301075269

In [49]:
bg.score(X_train,y_train)

0.9838565022421525

In [61]:
y_pred_class = bg.predict(X_train)
training_accuracy = metrics.accuracy_score(y_train, y_pred_class)
training_error = 1 - training_accuracy

y_pred_class = bg.predict(X_test)
testing_accuracy = metrics.accuracy_score(y_test, y_pred_class)
testing_error = 1 - testing_accuracy

print("Best Bagging Model Test Accuracy: {0}, Test Error: {1}".format(training_accuracy, training_error))
print("Best Bagging Model Test Accuracy: {0}, Test Error: {1}".format(testing_accuracy, testing_error))

Best Bagging Model Test Accuracy: 0.9838565022421525, Test Error: 0.016143497757847514
Best Bagging Model Test Accuracy: 0.8817204301075269, Test Error: 0.11827956989247312


In [62]:
metrics.confusion_matrix(y_test, y_pred_class)

array([[656,   6],
       [ 82,   0]])

In [236]:
report=classification_report(y_over_test, y_pred_class)
print(report)

              precision    recall  f1-score   support

           0       0.68      1.00      0.80       700
           1       0.83      0.04      0.08       350

    accuracy                           0.68      1050
   macro avg       0.75      0.52      0.44      1050
weighted avg       0.73      0.68      0.56      1050

